In [70]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import pickle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

sns.set()

In [15]:
df=pd.read_csv('Preprocessed.csv')

In [16]:
df.head(3)

,Group_4,Group_3,Group_2,Group_1,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Month_,WeekDay
0,1,0,0,0,26,2015-07-07,289,36,33,239.554,30,0,2,1,4,7,1
1,0,0,0,0,0,2015-07-14,118,13,50,239.554,31,0,1,0,0,7,1
2,0,1,0,0,23,2015-07-15,179,51,38,239.554,31,0,0,0,2,7,2


In [17]:
extra_absent=np.where(df['Absenteeism Time in Hours']>df['Absenteeism Time in Hours'].median(),1,0)

In [18]:
df['Extra_absent']=extra_absent

In [19]:
df=df.drop(columns=['Absenteeism Time in Hours','Date'])

In [20]:
check_point_1=df

In [24]:
x=df.iloc[:,:-1]
y=extra_absent

In [25]:

scaler=preprocessing.MinMaxScaler()
scaler.fit(x)
x_scaled=scaler.transform(x)

In [32]:
x_train,x_test,y_train,y_test=train_test_split(x_scaled,y,test_size=0.1,random_state=42)

In [34]:
reg=LogisticRegression()
reg.fit(x_train,y_train)
reg.score(x_test,y_test)


0.7

In [62]:
# Extracting the Coefficients of the trained model
reg_coef=pd.DataFrame()
names=x.columns.values
reg_coef['Name_of_feature']=names
reg_coef["Coef_"]=np.transpose(reg.coef_)

In [63]:
reg_coef.index=reg_coef.index +2

In [64]:
reg_coef.loc[1]=['Intercept_',reg.intercept_[0]]
reg_coef=reg_coef.sort_index()

In [65]:
reg_coef['Odds_ratio']=np.exp(reg_coef['Coef_'])

In [69]:
reg_coef.sort_values('Odds_ratio' , ascending=False)

,Name_of_feature,Coef_,Odds_ratio
5,Group_1,2.885380,17.910367
4,Group_2,2.305189,10.026073
7,Transportation Expense,2.137198,8.475656
13,Children,1.415716,4.119434
3,Group_3,1.386798,4.002016
2,Group_4,0.863475,2.371387
11,Body Mass Index,0.686870,1.987485
15,Month_,0.191006,1.210466
10,Daily Work Load Average,0.171429,1.187000
6,Reason for Absence,0.123391,1.131327


In [71]:
with open ('model', 'wb') as file:
    pickle.dump(reg,file)

In [72]:
with open ('scaler', 'wb') as file:
    pickle.dump(scaler,file)